In [1]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
import warnings
import sys
sys.path.append('../utils')
from treasury_base import *

warnings.filterwarnings("ignore")

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def direct_pred_retrieval():
    data = treasury_data_retrieval('us_treasury_rates_large.csv')
    data = data.set_index('Date')
    targets = data.columns

    # List of moving average windows
    window_list = [1, 3, 5]

    # List of lags to calculate moving average
    lag_list = [1]

    # List of future date values
    shift_list = [_ for _ in range(20)]

    # Generate future columns
    for shift in shift_list:
        for col in targets:
            data[f'{col}_+_{shift}'] = data[col].shift(-shift)

    # Generate past moving average columns
    for lag in lag_list:
        for window in window_list:
            for col in targets:
                data[f'{col}_-_{lag}_window_{window}'] = data[col].shift(1).rolling(window).mean()
    return data, targets

def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse ** 0.5

In [3]:
data, ori_col = direct_pred_retrieval()
all_cols = data.columns
data.tail()

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,...,3 Mo_-_1_window_5,6 Mo_-_1_window_5,1 Yr_-_1_window_5,2 Yr_-_1_window_5,3 Yr_-_1_window_5,5 Yr_-_1_window_5,7 Yr_-_1_window_5,10 Yr_-_1_window_5,20 Yr_-_1_window_5,30 Yr_-_1_window_5
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-02,4.75,4.63,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,...,4.608,4.444,4.360,4.222,4.202,4.160,4.214,4.282,4.546,4.466
2024-12-03,4.66,4.56,4.49,4.40,4.27,4.17,4.13,4.11,4.17,4.23,...,4.584,4.438,4.336,4.182,4.160,4.116,4.170,4.238,4.504,4.418
2024-12-04,4.65,4.54,4.47,4.38,4.23,4.13,4.09,4.07,4.13,4.19,...,4.558,4.426,4.316,4.174,4.144,4.104,4.162,4.230,4.498,4.408
2024-12-05,4.59,4.53,4.46,4.38,4.23,4.15,4.10,4.07,4.12,4.17,...,4.530,4.412,4.288,4.158,4.120,4.084,4.140,4.208,4.476,4.382
2024-12-06,4.57,4.50,4.42,4.34,4.19,4.10,4.05,4.03,4.09,4.15,...,4.502,4.402,4.266,4.150,4.106,4.076,4.130,4.192,4.458,4.360


In [6]:
# Out-of-sample test size, diff between sliding element = test size
test_size = 20
sliding_list = [0]

# Set variables for cross-validation
truth_df = pd.DataFrame()
naive_df = pd.DataFrame()
kan_df = pd.DataFrame()

# Loop over sliding windows
for sliding in sliding_list:
    
    # Trim original data by sliding window size
    df = data[:len(data)-sliding]

    # Use 2 years of data (500 days) for training
    df_train, df_test = df[-test_size-500:-test_size], df[-test_size:]
    len_train = len(df_train)

    # Append to truth dataframe
    truth_df = pd.concat([truth_df, df_test[ori_col]], axis=0, ignore_index=False)

    # Append to naive dataframe
    naive_element = pd.DataFrame([df_train[ori_col].iloc[-1].values] * test_size)
    naive_df = pd.concat([naive_df, naive_element], axis=0, ignore_index=True)

    # Initialize predictions array
    pred = list()

    for h in range(test_size):

        # Print checkpoints
        print(f'LAST DAY OF DATASET: {-sliding}, FUTURE STEPS: {h+1}')

        # If h = 0 target columns unchanged
        if h == 0:  
            target_col = ori_col
        # If h > 0 target columns modified
        else:       
            target_col = [f'{element}_+_{h}' for element in ori_col]
        
        # Extract feature columns
        feature_col = [element for element in all_cols if 'window' in element]

        # Cut train data due to direct forecast
        df_train_modified = df_train[:(len_train-h)]

        # Test data is the first row 
        df_test_modified = df_test.iloc[[0]]
        print(len(df_train_modified))

        X_train, y_train = df_train_modified[feature_col], df_train_modified[target_col]
        X_test, y_test = df_test_modified[feature_col], df_test.iloc[h][ori_col]

        n_inputs = X_train.shape[1]
        n_outputs = y_train.shape[1]

        dataset = dict()
        dtype = torch.get_default_dtype()
        dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
        dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
        dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
        dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)

        # Initialize the model
        model = KAN(width=[n_inputs, 32, n_outputs], grid=3, k=2, seed=42, device=device, symbolic_enabled=False, save_act=False, auto_save=False)

        # Train the model and compute metrics
        results = model.fit(dataset, opt="Adam", lr=0.001, steps=500, metrics=(train_mse, test_mse))

        # loss_fn = loss_fn_eval = lambda x, y: torch.mean((x - y) ** 2)
        # p = 
        # train_loss = loss_fn(p, dataset['train_label'])
        # print(train_loss)
        # torch.sqrt(train_loss).cpu().detach().numpy()


        pred.append(model.forward(dataset['test_input']).cpu().detach().numpy().flatten())
        # print(n_inputs, n_outputs)

    kan_element = pd.DataFrame(pred)
    kan_df = pd.concat([kan_element, kan_df], axis=0, ignore_index=True)

# df_train_modified
# df_test_modified
# y_train
# X_test
# model(dataset['test_input'])

LAST DAY OF DATASET: 0, FUTURE STEPS: 1
500


description:   0%|                                                          | 0/500 [00:00<?, ?it/s]

| train_loss: 8.35e-02 | test_loss: 3.40e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:15<00:00, 32.92


LAST DAY OF DATASET: 0, FUTURE STEPS: 2
499


| train_loss: 1.00e-01 | test_loss: 3.57e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:14<00:00, 33.44


...

| train_loss: 1.76e-01 | test_loss: 5.03e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:14<00:00, 33.65


LAST DAY OF DATASET: 0, FUTURE STEPS: 19
482


| train_loss: 1.76e-01 | test_loss: 4.77e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:14<00:00, 33.75


LAST DAY OF DATASET: 0, FUTURE STEPS: 20
481


| train_loss: 1.78e-01 | test_loss: 4.96e-01 | reg: 0.00e+00 | : 100%|█| 500/500 [00:14<00:00, 33.96


In [7]:
import pickle


dataframes = {
    "naive_df": naive_df,
    "kan_df": kan_df,
    "truth_df": truth_df
}

# Specify the file name
filename = f"dfs_{test_size}_steps_ahead.pkl"

# Pickle the DataFrames into a file
with open(filename, "wb") as file:
    pickle.dump(dataframes, file)

print(f"DataFrames have been saved to {filename}")

DataFrames have been saved to dfs_20_steps_ahead.pkl


In [10]:
naive_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
1,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
2,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
3,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
4,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
5,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
6,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
7,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
8,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6
9,4.68,4.71,4.64,4.41,4.31,4.27,4.2,4.27,4.37,4.42,4.71,4.6


In [66]:
real = df_test[ori_col].values
mean_squared_error(real, pred, squared=False)

np.float64(0.07110105203070181)

In [67]:
naive = [df_train[ori_col].iloc[-1].values] * test_size
mean_squared_error(real, naive, squared=False)

np.float64(0.05929291173832928)

In [43]:
df_test[ori_col]

,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,
2024-11-15,4.70,4.67,4.60,4.44,4.34,4.31,4.27,4.30,4.36,4.43,4.70,4.60
2024-11-18,4.70,4.65,4.63,4.44,4.33,4.29,4.25,4.28,4.35,4.42,4.70,4.61
2024-11-19,4.67,4.63,4.62,4.44,4.34,4.27,4.24,4.25,4.32,4.39,4.66,4.57
2024-11-20,4.68,4.63,4.62,4.44,4.37,4.31,4.26,4.28,4.34,4.41,4.66,4.59
2024-11-21,4.72,4.67,4.63,4.45,4.39,4.34,4.30,4.30,4.36,4.43,4.68,4.61


In [33]:
pred

[array([4.66407683, 4.6880482 , 4.58706215, 4.49451238, 4.26064394,
        4.201187  , 4.10178506, 4.08798675, 4.17354346, 4.25746796,
        4.51443222, 4.41495509]),
 array([4.65792723, 4.68638298, 4.58311401, 4.49485415, 4.25686574,
        4.20768825, 4.10373294, 4.08724269, 4.17113352, 4.25754362,
        4.50974496, 4.41236799]),
 array([4.65596465, 4.6844064 , 4.58216824, 4.49543679, 4.25406714,
        4.21491783, 4.10799169, 4.0888819 , 4.17270552, 4.26030764,
        4.50860762, 4.41186676]),
 array([4.65393193, 4.68071803, 4.58079666, 4.49318703, 4.24684322,
        4.21861217, 4.10869463, 4.0866514 , 4.17227466, 4.26165436,
        4.50830992, 4.41167705]),
 array([4.65191622, 4.67728895, 4.57925671, 4.49220682, 4.24048311,
        4.22277714, 4.11229057, 4.08584026, 4.17406209, 4.26400507,
        4.51049918, 4.41391378])]

In [35]:
naive

[array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36]),
 array([4.76, 4.69, 4.58, 4.42, 4.3 , 4.13, 4.1 , 4.05, 4.1 , 4.18, 4.45,
        4.36])]

In [44]:
real

array([[4.7 , 4.67, 4.6 , 4.44, 4.34, 4.31, 4.27, 4.3 , 4.36, 4.43, 4.7 ,
        4.6 ],
       [4.7 , 4.65, 4.63, 4.44, 4.33, 4.29, 4.25, 4.28, 4.35, 4.42, 4.7 ,
        4.61],
       [4.67, 4.63, 4.62, 4.44, 4.34, 4.27, 4.24, 4.25, 4.32, 4.39, 4.66,
        4.57],
       [4.68, 4.63, 4.62, 4.44, 4.37, 4.31, 4.26, 4.28, 4.34, 4.41, 4.66,
        4.59],
       [4.72, 4.67, 4.63, 4.45, 4.39, 4.34, 4.3 , 4.3 , 4.36, 4.43, 4.68,
        4.61]])

In [ ]:
for h in range(0, 5):
    if h == 0:
        target_col = ori_col
    else:
        target_col = [f'{element}_+_{h}' for element in ori_col]
    
    feature_col = [element for element in all_cols if 'window' in element]

    df_train_modified = df_train[:(len_train-h)]
    df_test_modified = df_test.iloc[[0]]
    print(len(df_train_modified))

    X_train, y_train = df_train_modified[feature_col], df_train_modified[target_col]
    X_test, y_test = df_test_modified[feature_col], df_test.iloc[h][ori_col]

X_test